# Mixtec Gender Classification

### Imports

In [1]:
# %matplotlib inline
# %pip install pandas numpy torcheval torch matplotlib tensorboard
import os

from pathlib import Path
import PIL
from PIL import Image

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import random_split, ConcatDataset
from torchvision import datasets, transforms
from torcheval.metrics import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.sampler import WeightedRandomSampler

torch.manual_seed(42)

/apps/pytorch/2.0.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define hardware

In [2]:
!pwd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}\n')
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

%tensorboard --logdir=runs --bind_all

/orange/ufdatastudios/christan/mixteclabeling/notebooks
Using device: cuda


NVIDIA A100-SXM4-80GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


UsageError: Line magic function `%tensorboard` not found.


### Define path to images

In [ ]:
basepath = Path('/home/alexwebber/toorange/alexwebber/mixteclabeling') # Base data directory
path_v = basepath / 'data/labeled_figures/codex_vindobonensis/gender/'
path_n = basepath / 'data/labeled_figures/codex_nuttall/gender/'
path_s = basepath / 'data/labeled_figures/codex_selden/gender/'

### Load figures into pandas, visualize

In [ ]:
# Random Block Transform
class AddRandomBlockNoise(torch.nn.Module):
    def __init__(self, n_k=8, size=64):
        super(AddRandomBlockNoise, self).__init__()
        self.n_k = int(n_k * np.random.rand()) # Random number of boxes
        self.size = int(size * np.random.rand()) # Max size
    
    def forward(self, tensor):
        h, w = self.size, self.size
        img = np.asarray(tensor)
        img_size_x = img.shape[1]
        img_size_y = img.shape[2]
        boxes = []
        for k in range(self.n_k):
            if (img_size_y >= h or img_size_x >=w): break
            print(f"{h=} {w=} {img_size_x=} {img_size_y=}")
            x = np.random.randint(0, img_size_x-w, 1)[0] # FIXME the shape may be zero
            y = np.random.randint(0, img_size_y-h, 1)[0]
            img[:, y:y+h, x:x+w] = 0
            boxes.append((x,y,h,w))
        #img = Image.fromarray(img.astype('uint8'), 'RGB')
        return torch.from_numpy(img)
    
    def __repr__(self):
        return self.__class__.__name__ + '(blocks={0}, size={1})'.format(self.n_k, self.size)

In [ ]:
## Load CSV
mixtec_figures = pd.read_csv(basepath / "data/mixtec_figures.csv")

print(mixtec_figures.groupby('quality')['gender'].value_counts())
print('\n')
print(mixtec_figures['gender'].value_counts())
print('\n')
print(mixtec_figures['quality'].value_counts())

## Load Tensorboard output
writer = SummaryWriter(log_dir='runs/mixtec_experiment_gender')


### Load figures into datasets by codex, apply transforms

In [ ]:
# Random Block Transform
class AddRandomBlockNoise(torch.nn.Module):
    def __init__(self, n_k=8, size=64):
        super(AddRandomBlockNoise, self).__init__()
        self.n_k = int(n_k * np.random.rand()) # Random number of boxes
        self.size = int(size * np.random.rand()) # Max size
    
    def forward(self, tensor):
        h, w = self.size, self.size
        img = np.asarray(tensor)
        img_size_x = img.shape[1]
        img_size_y = img.shape[2]
        boxes = []
        for k in range(self.n_k):
            if (img_size_y >= h or img_size_x >=w): break
            print(f"{h=} {w=} {img_size_x=} {img_size_y=}")
            x = np.random.randint(0, img_size_x-w, 1)[0] # FIXME the shape may be zero
            y = np.random.randint(0, img_size_y-h, 1)[0]
            img[:, y:y+h, x:x+w] = 0
            boxes.append((x,y,h,w))
        #img = Image.fromarray(img.astype('uint8'), 'RGB')
        return torch.from_numpy(img)
    
    def __repr__(self):
        return self.__class__.__name__ + '(blocks={0}, size={1})'.format(self.n_k, self.size)

In [ ]:
## Define image transforms
## List of transforms https://pytorch.org/vision/stable/auto_examples/plot_transforms.html
transform = transforms.Compose(
    [transforms.ToTensor(),
     AddRandomBlockNoise(),
     transforms.Resize((227, 227), antialias=True),
     # transforms.Grayscale(),
     
     #transforms.ColorJitter(contrast=0.5),
     #transforms.RandomRotation(360),     # Maybe useful for standng and sitting
     #transforms.RandomHorizontalFlip(50),
     #transforms.RandomVerticalFlip(50)
])

## Load images into PyTorch dataset
vindobonensis_dataset = datasets.ImageFolder(path_v, transform=transform)
nuttall_dataset = datasets.ImageFolder(path_n, transform=transform)
selden_dataset = datasets.ImageFolder(path_s, transform=transform)

### Concatenate datasets

In [ ]:
figures_dataset = ConcatDataset([vindobonensis_dataset, nuttall_dataset, selden_dataset])

print(figures_dataset)

### Assign classes to map

In [ ]:
class_map = {0: "female", 1: "male"}

### Print random image for sanity check

In [ ]:
# Access a random image from the dataset

for i in range(1):
    random_index = np.random.randint(len(figures_dataset))
    image, label = figures_dataset[random_index]

    # Convert the image tensor to a NumPy array and transpose it
    image = image.permute(1, 2, 0)
    image = image.numpy()

    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

### Visualize dataloaders

In [ ]:
def count_classes(dataset, n_classes=2):
    image_count = [0]*(n_classes)
    for img in dataset:
        image_count[img[1]] += 1
    return image_count

def sampler_(dataset, n_classes=2):
    dataset_counts = count_classes(dataset)
    num_samples = len(dataset_counts)
    labels = [tag for _,tag in dataset]

    class_weights = [num_samples/dataset_counts[i] for i in range(n_classes)]
    weights = [class_weights[labels[i]] for i in range(num_samples)]
    sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)
    return sampler

### Split combined dataset into training and testing sets and load into DataLoaders

In [ ]:
batch_size = 128

train_set, test_set = random_split(figures_dataset, [0.8, 0.2])

sampler = sampler_(train_set.dataset)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size,  shuffle = True)

# Training

### Define CNN

In [ ]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(16 * 56 * 56, 1568)  # Adjusted size
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)
        x = F.relu(x)
        
        x = self.dropout1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 16 * 56 * 56)
        
        x = self.dropout2(x)
        x = self.fc1(x)
        x = x.view(batch_size, -1)
        return x




### Create the model

In [ ]:
# Create the model
cnn = CNN()
cnn.to(device)

### Define hyperparameters

In [ ]:
lossweight = torch.tensor([1.5,1.0]).to(device)
#lossweight.to(device)
criterion = torch.nn.CrossEntropyLoss()
#criterion = torch.nn.NLLLoss()
#optimizer = torch.optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(cnn.parameters())

### Train model

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    losses = []
    
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        # forward
        outputs = cnn(inputs)
        
        # metrics
        train_loss = criterion(outputs, labels)
        #train_accuracy = torch.sum(outputs == labels)
        running_loss += train_loss.item()
        
        # backward
        optimizer.zero_grad()
        train_loss.backward()
        
        # gradient descent or adam step
        optimizer.step()

        # print statistics
        
        if i % batch_size == 0:
            print("Epoch: " + str(epoch + 1) + " | " "Loss: " + str(running_loss))
            
            # write to TensorBoard
            writer.add_scalar('Loss/train', losses[n_iter], n_iter)
        
            running_loss = 0.0

print('Finished Training')

### Write to TensorBoard

In [ ]:
cnn.eval()

### View incorrectly labeled samples

In [ ]:
for i, data in enumerate(train_loader, 0):
    images, labels = data[0].to(device), data[1].to(device)
    
    outputs = cnn(images)
    
    _, predictions = torch.max(outputs, 1)
    
    correct = 0
    total = len(predictions)
#     for label, image, prediction in zip(labels, images, predictions):
#         if label != prediction:
#             image = image.permute(1, 2, 0)
#             image = image.cpu().numpy()

#             plt.imshow(image)
#             plt.title("Prediction: " + class_map[prediction.item()] + " | Label: " + class_map[label.item()])
#             plt.axis('off')
#             plt.show()
            
    
            

### Save model

In [ ]:
savepath = "../models/mixtec_gender_classifier.pth"

torch.save(cnn.state_dict(), savepath)

## Visualize learning

### Define tensorboard output functions

In [ ]:
import io

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
def gen_plot(img):
    img = img.mean(dim=0)
    img = img / 2 + 0.5
    npimg = img.numpy()
    
    
    plt.figure()
    plt.imshow(npimg, cmap="inferno")
    buf = io.BytesIO()
    plt.savefig(buf, format='jpeg')
    buf.seek(0)
    
    return buf

### Output sample heatmap of selected features

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

image_grid = torchvision.utils.make_grid(images)

# Prepare the plot
plot_buf = gen_plot(image_grid)

image = Image.open(plot_buf)
image = transforms.ToTensor()(image).unsqueeze(0)

#img_grid = torchvision.utils.make_grid(images)

# show images
# matplotlib_imshow(image, one_channel=True)

# Testing

### Load images and labels from test_loader

In [ ]:
data_iter = iter(test_loader)
images, labels = next(data_iter)

### Load model

In [ ]:
cnn = CNN()
cnn.load_state_dict(torch.load(savepath))
cnn.to(device)

### Produce predictions and calculate accuracy of model

In [ ]:
cnn.eval()

predicted_list = []
target_list = []

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        target_list += labels.cpu()
        
        # calculate outputs by running images through the network
        outputs = cnn(images)
        
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        print(_)
        predicted_list += predicted
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {str(len(test_set))} test images: {100 * correct // total} %')

In [ ]:
print(f"Predicted: {torch.tensor(predicted_list)}")
print(f"Truth    : {torch.tensor(target_list)}")

metric_names = ["Accuracy", "Precision", "Recall", "F1"]
metrics = [BinaryAccuracy(), BinaryPrecision(), BinaryRecall(), BinaryF1Score()]

for metric, name in zip(metrics, metric_names):
    metric.update(torch.tensor(predicted_list), torch.tensor(target_list))
    print(f"{name:<9}: {metric.compute()}")